In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
%matplotlib inline
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np

import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split, KFold
from sklearn.grid_search import GridSearchCV

import transform_utils as tu
from kaggle_helper import KaggleHelper

In [8]:
kh = KaggleHelper("submissions.db", "predictions")

table submissions already exists
table validation_scores already exists


In [40]:
%aimport transform_utils
%aimport kaggle_helper

In [41]:
%aimport

Modules to reload:
kaggle_helper transform_utils

Modules to skip:



In [42]:
raw = pd.read_csv("data/train.csv")
print("Unique UPC:", len(raw.Upc.unique()))
print("Unique DepartmentDescription:", len(raw.DepartmentDescription.unique()))
print("Unique FinelineNumber:", len(raw.FinelineNumber.unique()))
raw.head()

Unique UPC: 97715
Unique DepartmentDescription: 69
Unique FinelineNumber: 5196


,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [71]:
start = datetime.datetime.now()
%autoreload
group_cols = ['DepartmentDescription', 'Weekday']
keep_cols = ['VisitNumber', 'ScanCount', 'Returns']
df = tu.preprocess_walmart(raw, group_cols, keep_cols, True)

print("Total Time:", (datetime.datetime.now() - start).total_seconds())

Total Time: 120.771017


In [74]:
def fix_index(idx_val):
    top, bottom = idx_val
    if top == "TripType":
        return "TripType"
    else:
        return ''.join(idx_val)

In [76]:
df.columns = list(map(fix_index, df.columns.tolist())) # if hierarchical...

In [80]:
X = df.reset_index().drop("TripType", axis=1).drop("VisitNumber", axis=1)
sorted_columns = sorted(X.columns)
X = X[sorted_columns]
y = df.TripType

In [81]:
XTrain, XVal, yTrain, yVal = train_test_split(X.values, y.values, test_size=0.3, random_state=200)

In [82]:
search_grid = {
    "n_estimators": [130],
    "max_features": ["auto"],
    "min_samples_split": [10],
    "min_samples_leaf": [1]
}

In [83]:
start = datetime.datetime.now()
clf = RandomForestClassifier()

model = GridSearchCV(clf, search_grid, n_jobs=3)
model.fit(X, y)
score = model.score(XVal, yVal)
print(score)

print("Total Time:", (datetime.datetime.now() - start).total_seconds())

0.889001149706
Total Time: 182.472035


In [84]:
kh.save_validation_score(model, score, len(XTrain))

Enter Notes to save to DB: improved training, and indexing


# Only Kaggle Exists Below Here

In [85]:
%autoreload
kaggleTestRaw = pd.read_csv("data/test.csv")
keep_cols = ['VisitNumber', 'ScanCount', 'Returns']
kaggleTest = tu.preprocess_walmart(kaggleTestRaw, group_cols, keep_cols)
kaggleTest.columns = list(map(fix_index, kaggleTest.columns.tolist()))

In [86]:
missing_columns = set(sorted_columns).difference(set(kaggleTest.columns))
print("missing columns")
print("-"*15)
for col in missing_columns:
    print(col)
    kaggleTest[col] = 0
kaggleTest = kaggleTest[sorted_columns]
assert list(X.columns) == list(kaggleTest.columns)

missing columns
---------------
DepartmentDescription_HEALTH AND BEAUTY AIDScount_nonzero
DepartmentDescription_HEALTH AND BEAUTY AIDSsum


In [87]:
%autoreload
test_predictions = model.predict(kaggleTest.values)
output = tu.convert_predictions(test_predictions, 
                               output_index=pd.Series(kaggleTest.index), 
                               actual_trip_types=df.TripType.unique())

Missing Categories {14}


In [88]:
kh.save_test_predictions(output, model, len(XTrain))

Enter Notes to save to DB: improved training, and indexing, apply 2 functions
